In [4]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=3f18fd946e5ffbb31d634830c81309b5d24f69cca08593cf20d53d15308e6342
  Stored in directory: /Users/dheeraj/Library/Caches/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
DEPRECATION: feast 0.12.1 has a non-standard dependency specifier PyYAML>=5.4.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of feast or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: feast 0.12.1 has a non-standard dependency specifier tenacity>=7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of feast or contact the author to sugge

In [10]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import *
import pandas as pd
from time import perf_counter

In [7]:

spark = SparkSession.builder.appName('learn').getOrCreate()

24/02/20 18:10:27 WARN Utils: Your hostname, Dheerajs-MacBook-Air-1392.local resolves to a loopback address: 127.0.0.1; using 10.0.0.207 instead (on interface en0)
24/02/20 18:10:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 18:10:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df = spark.range(0, 10 * 1000 * 1000).withColumn('id', (col('id') / 10000).cast('integer')).withColumn('v', rand())
df.cache()
df.count()

df.show()

+---+--------------------+
| id|                   v|
+---+--------------------+
|  0|  0.7019090957422275|
|  0|  0.9933700390740879|
|  0| 0.09645608750983481|
|  0|  0.9110235501877338|
|  0|  0.8890663311497601|
|  0| 0.05317098055069769|
|  0|0.042303986602792776|
|  0|  0.6647547444772897|
|  0| 0.32842395299411886|
|  0|0.018639868702528317|
|  0|  0.6313347143133267|
|  0|  0.0831228330396655|
|  0|  0.4276704236404433|
|  0| 0.25428379813778945|
|  0|  0.8362118971678169|
|  0|  0.8267217422966622|
|  0|  0.7363125165408233|
|  0| 0.09248071191481433|
|  0|  0.7114607813627912|
|  0| 0.43602389756778626|
+---+--------------------+
only showing top 20 rows



In [11]:
@udf('double')
def plus_one(v):
    return v + 1

%timeit df.withColumn('v', plus_one(df.v)).agg(count(col('v'))).show()

+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+

3.76 s ± 207 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
import pandas as pd

@pandas_udf("double")
def pandas_plus_one(v: pd.Series) -> pd.Series:
    return v + 1

%timeit df.withColumn('v', pandas_plus_one(df.v)).agg(count(col('v'))).show()

+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+



+--------+
|count(v)|
+--------+
|10000000|
+--------+

1.54 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
from scipy import stats

@udf('double')
def cdf(v):
    return float(stats.norm.cdf(v))

%timeit df.withColumn('cumulative_probability', cdf(df.v)).agg(count(col('cumulative_probability'))).show()

+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+

1min 54s ± 13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
@pandas_udf('double')
def pandas_cdf(v: pd.Series) -> pd.Series:
    return pd.Series(stats.norm.cdf(v))

%timeit df.withColumn('cumulative_probability', pandas_cdf(df.v)).agg(count(col('cumulative_probability'))).show()

+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+



+-----------------------------+
|count(cumulative_probability)|
+-----------------------------+
|                     10000000|
+-----------------------------+

2.4 s ± 604 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
from pyspark.sql import Row
@udf(ArrayType(df.schema))
def substract_mean(rows):
    vs = pd.Series([r.v for r in rows])
    vs = vs - vs.mean()
    return [Row(id=rows[i]['id'], v=float(vs[i])) for i in range(len(rows))]
  
%timeit df.groupby('id').agg(collect_list(struct(df['id'], df['v'])).alias('rows')).withColumn('new_rows', substract_mean(col('rows'))).withColumn('new_row', explode(col('new_rows'))).withColumn('id', col('new_row.id')).withColumn('v', col('new_row.v')).agg(count(col('v'))).show()


24/02/20 18:29:00 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:00 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:00 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:09 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:14 WARN TaskMemoryManager: Failed to allocate a page (2097136 bytes), try again.
24/02/20 18:29:15 WARN TaskMemoryManager

KeyboardInterrupt: 

24/02/20 18:34:46 ERROR Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
Exception in thread "stdout writer for python3" java.lang.OutOfMemoryError: Java heap space
24/02/20 18:34:55 ERROR Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
24/02/20 18:34:55 ERROR Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
Exception in thread "stdout writer for python3" Exception in thread "stdout writer for python3" 24/02/20 18:34:55 ERROR Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
Exception in thread "stdout writer for python3" 24/02/20 18:34:55 ERROR Utils: Uncaught exception in thread stdout writer for python3
java.lang.OutOfMemoryError: Java heap space
Exception in thread "stdout writer for python3" java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryE

In [ ]:
def subtract_mean(pdf: pd.DataFrame) -> pd.DataFrame:
    return pdf.assign(v=pdf.v - pdf.v.mean())
%timeit df.groupby('id').applyInPandas(subtract_mean, schema=df.schema).agg(count(col('v'))).show()

In [ ]:
df2 = df.withColumn('y', rand()).withColumn('x1', rand()).withColumn('x2', rand()).select('id', 'y', 'x1', 'x2')
df2.show()                                                               

In [ ]:
import statsmodels.api as sm
# df has four columns: id, y, x1, x2
group_column = 'id'
y_column = 'y'
x_columns = ['x1', 'x2']
schema = df2.select(group_column, *x_columns).schema
# Input/output are both a pandas.DataFrame
def ols(pdf: pd.DataFrame) -> pd.DataFrame:
    group_key = pdf[group_column].iloc[0]
    y = pdf[y_column]
    X = pdf[x_columns]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return pd.DataFrame([[group_key] + [model.params[i] for i in   x_columns]], columns=[group_column] + x_columns)
beta = df2.groupby(group_column).applyInPandas(ols, schema=schema)
beta.show()
